In [95]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import re
import pandas as pd
import numpy as np

In [2]:
def cantidad_fechas(driver):

    elem = driver.find_element(By.ID, "fixtseccion")
    cant_fechas = len(elem.find_elements(By.XPATH, '//div[@class="cfecha"]'))+1
    
    return driver, cant_fechas

In [3]:
def get_links(driver):
    
    elem = driver.find_element(By.ID, "fixtseccion")
    filter = f'//div[@class="cfecha"][text()="{i}"]'
    link = elem.find_element(By.XPATH, filter)
    link.click()
    time.sleep(1)

    elems = driver.find_element(By.ID, "fixturein")
    elems = elems.find_elements(By.XPATH, ".//a[@href]")
    links = [elem.get_attribute("href") for elem in elems]
    
    return driver, links

In [4]:
def pretify_text(text):
    return text.replace("         ", "").replace("        ", "").replace("       ", "").replace("\n", "").replace("%", "")

In [73]:
def tarjetas(soup):
    regex = re.compile('.*images/amarilla.png.*')
    amarillas_local = soup.find("table", {"id": ["formacion1"]}).findAll("img", {"src": [regex]})
    if len(amarillas_local) > 0:
        amarillas_local = len(amarillas_local) - 1
    else:
        amarillas_local = 0
    amarillas_visitante = soup.find("table", {"id": ["formacion2"]}).findAll("img", {"src": [regex]})
    if len(amarillas_visitante) > 0:
        amarillas_visitante = len(amarillas_visitante) - 1
    else:
        amarillas_visitante = 0

    regex = re.compile('.*images/roja.png.*')
    rojas_local = soup.find("table", {"id": ["formacion1"]}).findAll("img", {"src": [regex]})
    if len(rojas_local) > 0:
        rojas_local = len(rojas_local) - 1
    else:
        rojas_local = 0
    rojas_visitante = soup.find("table", {"id": ["formacion2"]}).findAll("img", {"src": [regex]})
    if len(rojas_visitante) > 0:
        rojas_visitante = len(rojas_visitante) - 1
    else:
        rojas_visitante = 0
    
    return amarillas_local, amarillas_visitante, rojas_local, rojas_visitante

In [74]:
def info(soup):
    data_local = soup.findAll("div", {"id": ["porcentaje1"]})
    posesion_local = pretify_text(data_local[0].get_text())
    tiros_arco_local = pretify_text(data_local[1].get_text())
    intentos_local = pretify_text(data_local[2].get_text())
    fouls_local = pretify_text(data_local[3].get_text())
    corners_local = pretify_text(data_local[4].get_text())

    data_visitante = soup.findAll("div", {"id": ["porcentaje2"]})
    posesion_visitante = pretify_text(data_visitante[0].get_text())
    tiros_arco_visitante = pretify_text(data_visitante[1].get_text())
    intentos_visitante = pretify_text(data_visitante[2].get_text())
    fouls_visitante = pretify_text(data_visitante[3].get_text())
    corners_visitante = pretify_text(data_visitante[4].get_text())

    return posesion_local, tiros_arco_local, intentos_local, fouls_local, corners_local, posesion_visitante, tiros_arco_visitante, intentos_visitante, fouls_visitante, corners_visitante

In [75]:
def equipos(soup):
    equipos = soup.findAll("td", {"class": ["nomequipo"]})
    equipo_local = pretify_text(equipos[0].get_text())
    equipo_visitante = pretify_text(equipos[1].get_text())

    return equipo_local, equipo_visitante

In [91]:
def goles(soup):
    goles_local = soup.find("div", {"id": ["ficha-resultado1"]}).get_text().replace(" ", "").replace("\n", "")
    goles_visitante = soup.find("div", {"id": ["ficha-resultado2"]}).get_text().replace(" ", "").replace("\n", "")

    return goles_local, goles_visitante

In [92]:
promiedos = []

torneos = [
    #"https://www.promiedos.com.ar/torneo=1009",
    "https://www.promiedos.com.ar/torneo=1008",
    #"https://www.promiedos.com.ar/torneo=1007",
    #"https://www.promiedos.com.ar/torneo=1097",
    #"https://www.promiedos.com.ar/primera"
]

for torneo in torneos:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(torneo)

    nombre_torneo = driver.find_element(By.ID, "titulo1").text
    print("Nombre del torneo:", nombre_torneo)

    driver, cant_fechas = cantidad_fechas(driver)
    print("-Cantidad de fechas a escrapear:", cant_fechas)

    for i in range(1, cant_fechas+1):
        driver, links = get_links(driver)
        if i == 1:
            print("--Cantidad de partidos por fecha:", len(links))

        print("-Scrapeando fecha", i, " de", cant_fechas)

        j = 1
        for link in links:
            print("--Scrapeando partido", j, " de", len(links))

            page = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'})
            soup = BeautifulSoup(page.content, "html.parser")
            soup = BeautifulSoup(soup.prettify(), "html.parser")

            equipo_local, equipo_visitante = equipos(soup)

            goles_local, goles_visitante = goles(soup)

            posesion_local, tiros_arco_local, intentos_local, fouls_local, corners_local, posesion_visitante, tiros_arco_visitante, intentos_visitante, fouls_visitante, corners_visitante = info(soup)

            amarillas_local, amarillas_visitante, rojas_local, rojas_visitante = tarjetas(soup)

            promiedos.append({
                "torneo": nombre_torneo,
                "fecha": i,
                "partido": j,
                "equipo_local": equipo_local,
                "equipo_visitante": equipo_visitante,
                "goles_local": goles_local,
                "goles_visitante": goles_visitante,
                "posesion_local": posesion_local, 
                "tiros_arco_local": tiros_arco_local, 
                "intentos_local": intentos_local, 
                "fouls_local": fouls_local, 
                "corners_local": corners_local, 
                "posesion_visitante": posesion_visitante, 
                "tiros_arco_visitante": tiros_arco_visitante, 
                "intentos_visitante": intentos_visitante, 
                "fouls_visitante": fouls_visitante, 
                "corners_visitante": corners_visitante,
                "amarillas_local": amarillas_local, 
                "amarillas_visitante": amarillas_visitante, 
                "rojas_local": rojas_local, 
                "rojas_visitante": rojas_visitante
            })

            j += 1        

C:\Users\camus\AppData\Local\Temp\ipykernel_24200\948456270.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Nombre del torneo: Campeonato 2018/19
-Cantidad de fechas a escrapear: 25
--Cantidad de partidos por fecha: 13
-Scrapeando fecha 1  de 25
--Scrapeando partido 1  de 13
--Scrapeando partido 2  de 13
--Scrapeando partido 3  de 13
--Scrapeando partido 4  de 13
--Scrapeando partido 5  de 13
--Scrapeando partido 6  de 13
--Scrapeando partido 7  de 13
--Scrapeando partido 8  de 13
--Scrapeando partido 9  de 13
--Scrapeando partido 10  de 13
--Scrapeando partido 11  de 13
--Scrapeando partido 12  de 13
--Scrapeando partido 13  de 13
-Scrapeando fecha 2  de 25
--Scrapeando partido 1  de 13
--Scrapeando partido 2  de 13
--Scrapeando partido 3  de 13
--Scrapeando partido 4  de 13
--Scrapeando partido 5  de 13
--Scrapeando partido 6  de 13
--Scrapeando partido 7  de 13
--Scrapeando partido 8  de 13
--Scrapeando partido 9  de 13
--Scrapeando partido 10  de 13
--Scrapeando partido 11  de 13
--Scrapeando partido 12  de 13
--Scrapeando partido 13  de 13
-Scrapeando fecha 3  de 25
--Scrapeando partido

In [93]:
df = pd.DataFrame(promiedos)
df.head()

,torneo,fecha,partido,equipo_local,equipo_visitante,goles_local,goles_visitante,posesion_local,tiros_arco_local,intentos_local,...,corners_local,posesion_visitante,tiros_arco_visitante,intentos_visitante,fouls_visitante,corners_visitante,amarillas_local,amarillas_visitante,rojas_local,rojas_visitante
0,Campeonato 2018/19,1,1,Velez,Newells,2,0,46,4,9,...,6,54,2,4,13,4,2,3,0,0
1,Campeonato 2018/19,1,2,Tigre,San Lorenzo,2,2,52,4,13,...,4,48,7,15,13,4,2,2,0,0
2,Campeonato 2018/19,1,3,Gimnasia (LP),Argentinos,1,0,33,1,1,...,3,67,3,11,16,4,3,2,0,0
3,Campeonato 2018/19,1,4,Union,Aldosivi,1,0,50,3,12,...,11,50,0,4,17,2,1,2,0,0
4,Campeonato 2018/19,1,5,Patronato,Colon,0,0,45,4,14,...,9,55,5,16,13,6,1,2,0,0


In [100]:
df.goles_local = df.goles_local.astype(int)
df.goles_visitante = df.goles_visitante.astype(int)

In [101]:
df["resultado"] = np.where(df.goles_local > df.goles_visitante, "L", np.where(df.goles_local == df.goles_visitante, "E", "V"))
df["puntos_local"] = np.where(df.goles_local > df.goles_visitante, 3, np.where(df.goles_local == df.goles_visitante, 1, 0))
df["puntos_visitante"] = np.where(df.goles_local > df.goles_visitante, 0, np.where(df.goles_local == df.goles_visitante, 1, 3))

In [105]:
locales = df.groupby(["equipo_local"])[["goles_local", "puntos_local"]].sum().reset_index().rename(columns = {"equipo_local": "equipo"})
locales

,equipo,goles_local,puntos_local
0,Aldosivi,16,21
1,Argentinos,5,9
2,Atl Tucuman,20,22
3,Banfield,11,15
4,Belgrano,12,18
5,Boca Juniors,22,28
6,Colon,13,19
7,Def y Justicia,20,25
8,Estudiantes (LP),12,19
9,Gimnasia (LP),15,23


In [106]:
visitantes = df.groupby(["equipo_visitante"])[["goles_visitante", "puntos_visitante"]].sum().reset_index().rename(columns = {"equipo_visitante": "equipo"})
visitantes

,equipo,goles_visitante,puntos_visitante
0,Aldosivi,5,12
1,Argentinos,10,13
2,Atl Tucuman,16,20
3,Banfield,16,14
4,Belgrano,4,6
5,Boca Juniors,20,23
6,Colon,8,4
7,Def y Justicia,13,28
8,Estudiantes (LP),9,10
9,Gimnasia (LP),6,6


In [107]:
posiciones = pd.merge(left=locales, right=visitantes, on="equipo")

In [110]:
posiciones["puntos_total"] = posiciones.puntos_local + posiciones.puntos_visitante
posiciones.sort_values("puntos_total", ascending=False)

,equipo,goles_local,puntos_local,goles_visitante,puntos_visitante,puntos_total
16,Racing Club,20,33,23,24,57
7,Def y Justicia,20,25,13,28,53
5,Boca Juniors,22,28,20,23,51
17,River Plate,22,23,19,22,45
2,Atl Tucuman,20,22,16,20,42
25,Velez,19,27,15,13,40
12,Independiente,20,23,15,15,38
23,Tigre,20,16,19,20,36
11,Huracan,16,20,12,14,34
13,Lanus,18,23,9,11,34
